(07/06/2023) Testing using neural nets to do regression. In particular, we are interested in learning:
$$
    f(v) = \mathbb{E}[\mathbf{X}^T\mathbf{K}\mathbf{Y} | V = v]
$$ where $v = v(\mathbf{X})$ is the energy of the states $\mathbf{X}$. The Monte Carlo data for $\mathbf{X}^T\mathbf{K}\mathbf{Y}$ and $v(\mathbf{X})$ are already generated. We attempt to learn this function by using an NN to minimize the MSE.

* We attempt to use Fourier-embedded neural network to perform the regression.

In [ ]:
from PINN.PhysicsInformedROPDF import *
from PINN.utils.dnn import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

In [ ]:
# load data
v_path = "../data/LinearOscillator/OU_noise_energy.mat"
cond_exp_path = "../data/LinearOscillator/cond_exp.mat"
data1 = scipy.io.loadmat(v_path)
data2 = scipy.io.loadmat(cond_exp_path)

In [ ]:
idx_mc = 1000
x = data1["v_data"][0:idx_mc, :]
y = data2["cond_exp_data"][0:idx_mc, :]
t = data1["tspan"].flatten()
plt.figure(1);
idx_time = 10
plt.scatter(x[:, idx_time], y[:, idx_time], s=0.2, color="red", alpha=1.0);
plt.xlabel(r"$V$"); plt.ylabel(r"$\mathbb{E}[X^TKY|V=v]$");

In [ ]:
# create time dependent dataset
nt = 1000
tgrid = t[:nt]
nx = x.shape[0]
X_data = []
y_data = []
for i in range(nt):
    t_i = tgrid[i]
    # append time to all points in x
    inputs_t = torch.tensor([t_i]).repeat(nx).reshape(-1, 1)
    inputs_x = torch.tensor(x[:, i]).reshape(-1, 1)
    outputs = torch.tensor(y[:, i]).reshape(-1, 1)
    X_i = torch.concat([inputs_t, inputs_x], dim=1)
    # append to full data
    X_data.append(X_i)
    y_data.append(outputs)
X_data = torch.concat(X_data)
y_data = torch.concat(y_data)

def train(inputs, outputs, model, optim, scheduler, batch_size, epochs, shuffle=True):
    X, y = inputs, outputs
    nx = X.shape[0]
    num_batches = int(nx/batch_size)
    all_losses = []
    for i in range(epochs):
        print("============================================================\n")
        print("Epoch = {}\n".format(i+1));
        print("============================================================\n")
        model.train()
        if shuffle:
            tmp = np.random.permutation(nx)
            X, y = X[tmp, :].data.clone(), y[tmp, :].data.clone()
        for idx in range(num_batches):
            if idx % 100 == 0:
                print("| => | Batch {} |\n".format(idx+1))
        # closure definition
            def closure():
                optim.zero_grad()
                start_idx = idx*batch_size
                end_idx = (idx+1)*batch_size
                if idx + 1 == num_batches:
                    # if last batch
                    end_idx = -1
                Xb, yb = X[start_idx:end_idx, :].data.clone(), y[start_idx:end_idx, :].data.clone()

                # require gradients
                Xb.requires_grad = True
                # make a prediction on the batch
                y_pred = model.forward(Xb)
                # compute L^2 loss
                loss = torch.mean((y_pred - yb)**2)
                # backpropagate
                loss.backward()
                if idx % 100 == 0:
                    print("==> Batch {} loss = {}".format(idx, loss.item()))
                all_losses.append(loss.item())
                return loss
            optim.step(closure=closure)
        if scheduler:
            # step scheduler after epoch if there is one
            scheduler.step()
            print("---------- \n")
            print("++ Learning rate reduced, now at = {}".format(scheduler.get_last_lr()[0]))
    return all_losses

In [ ]:
# train using scatter data with Fourier net
nn_fourier2d_cartesian = FourierProductEmbeddedDNN2d(
    layers_time=[40, 100, 100, 1], 
    layers_space=[40, 100, 100, 1], 
    activation=torch.nn.Tanh, 
    last_layer_activation=None, 
    mt=20, 
    mx=20, 
    freq_stds_t=[1.,2.,10.,20.,50.], 
    freq_stds_x=[1.,2.,10.]
)
optim = torch.optim.Adam(
    nn_fourier2d_cartesian.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
loss_fourier = train(X_data, y_data, nn_fourier2d_cartesian, optim, scheduler, 2**9, 30, shuffle=True)

In [ ]:
# save loss curve
scipy.io.savemat("./fig/cond_exp_examples/oscillator/fourier_nn_training_loss.mat", {"loss": loss_fourier})

In [ ]:
# predict the function we learned at regular grid
xgrid = np.linspace(0, x.max(), 2*nx)
xgrid = torch.tensor(xgrid)
tgrid = torch.tensor(t[:nt])
# cartesian grid
X = cartesian_data(tgrid, xgrid)
# predict
y_pred = nn_fourier2d_cartesian(X)

In [ ]:
# visualize contour
import matplotlib
y_pred2d = y_pred.reshape(2*nx, nt).detach().numpy().T

In [ ]:
# load prediction data
plot_data = scipy.io.loadmat("./fig/cond_exp_examples/oscillator/fourier_nn_predictions.mat")
y_pred2d = plot_data["y_pred"]
plt.figure(1, figsize=(8, 6));
font = {'size'   : 16}
matplotlib.rc('font', **font)
plt.pcolormesh(tgrid, xgrid, y_pred2d.T); plt.colorbar();
plt.xlabel(r"Time $t$"); plt.ylabel(r"Energy $v$");
plt.title("Fourier NN");
plt.savefig("./fig/cond_exp_examples/oscillator/fourier_nn_predictions.png", dpi=100);

In [ ]:
# save predictions
scipy.io.savemat("./fig/cond_exp_examples/oscillator/fourier_nn_predictions.mat",\
                 {"xgrid": xgrid.detach().numpy(), 
                  "tgrid": tgrid.detach().numpy(), "y_pred": y_pred2d})

In [ ]:
# train using vanilla net
nn_vanilla2d = DNN(layers=[2, 100, 100, 1])
optim = torch.optim.Adam(
    nn_vanilla2d.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
loss_vanilla = train(X_data, y_data, nn_vanilla2d, optim, scheduler, 2**9, 30, shuffle=True)

In [ ]:
# save loss curve
scipy.io.savemat("./fig/cond_exp_examples/oscillator/vanilla_nn_training_loss.mat", {"loss": loss_vanilla})

In [ ]:
# predict the function we learned at regular grid
xgrid = np.linspace(0, x.max(), 2*nx)
xgrid = torch.tensor(xgrid)
tgrid = torch.tensor(t[:nt])
# cartesian grid
X = cartesian_data(tgrid, xgrid)
# predict
y_pred = nn_vanilla2d(X)

In [ ]:
# visualize contour
import matplotlib
plt.figure(1, figsize=(8, 6));
font = {'size'   : 16}
matplotlib.rc('font', **font)
y_pred2d = y_pred.reshape(2*nx, nt).detach().numpy().T
plt.pcolormesh(tgrid, xgrid, y_pred2d.T); plt.colorbar();
plt.xlabel(r"Time $t$"); plt.ylabel(r"Energy $v$");
plt.title("Vanilla NN");
plt.savefig("./fig/cond_exp_examples/oscillator/vanilla_nn_predictions.png", dpi=100);

In [ ]:
# save predictions
scipy.io.savemat("./fig/cond_exp_examples/oscillator/vanilla_nn_predictions.mat",\
                 {"xgrid": xgrid.detach().numpy(), 
                  "tgrid": tgrid.detach().numpy(), "y_pred": y_pred2d})

In [ ]:
# plot losses
import matplotlib
vanilla_loss = scipy.io.loadmat("./fig/cond_exp_examples/oscillator/vanilla_nn_training_loss.mat")["loss"].flatten()
fourier_loss = scipy.io.loadmat("./fig/cond_exp_examples/oscillator/fourier_nn_training_loss.mat")["loss"].flatten()
# plot
plt.figure(2, figsize=(20, 8));
font = {'size'   : 20}
matplotlib.rc('font', **font)
plt.plot(np.log10(vanilla_loss), lw=1.5, color='red', label="Vanilla");
plt.plot(np.log10(fourier_loss), lw=1.5, color='green', label="Fourier");
plt.xlabel("Iteration");
plt.ylabel("Batch MSE (Log Scale)");
plt.legend();
# save loss figure
plt.savefig("./fig/cond_exp_examples/oscillator/fourier_vanilla_loss_compare.png", dpi=100);

In [ ]:
# load density
f_v = data1["v_density"].T
density_dt = data1["dt"].flatten()[0]
density_tgrid = data1["tspan"].flatten()
density_xgrid = data1["xi"].flatten()
density_dx = density_xgrid[1]-density_xgrid[0]
density_nx = len(density_xgrid)
density_nt = len(density_tgrid)
# compute mean
all_means = np.zeros(density_nt)
for i in range(density_nt):
    all_means[i] = np.trapz(density_xgrid*f_v[i, :], density_xgrid)
# central differencing in time
central_diff_velocity = (all_means[1:]-all_means[0:-1])/(2*density_dt)

# load nn predictions
fourier_nn_pred_data = scipy.io.loadmat("./fig/cond_exp_examples/oscillator/fourier_nn_predictions.mat")
vanilla_nn_pred_data = scipy.io.loadmat("./fig/cond_exp_examples/oscillator/vanilla_nn_predictions.mat")
nn_tgrid = fourier_nn_pred_data["tgrid"].flatten()
nn_xgrid = fourier_nn_pred_data["xgrid"].flatten()

# fourier nn as a function of time (assume spatially constant)
fourier_nn_velocity = fourier_nn_pred_data["y_pred"][:, -1]
vanilla_nn_velocity = vanilla_nn_pred_data["y_pred"][:, -1]


# plot all
import matplotlib
fig = plt.figure(2, (10, 6));
font = {'size'   : 20}
matplotlib.rc('font', **font)
plt.plot(nn_tgrid, fourier_nn_velocity, lw=2.0, color="blue", label="FDNN");
plt.plot(nn_tgrid, vanilla_nn_velocity, lw=2.0, color="red", label="DNN");
# find appropriate index at t = 2
plot_t = 2.0
idx_t = int(plot_t/density_dt)
plt.plot(density_tgrid[0:idx_t], central_diff_velocity[0:idx_t], "--", color="black", lw=5.0, 
         alpha=0.25, label="Truth");
plt.legend();
plt.xlabel(r"Time $t$");
plt.ylabel(r"$Y_t^TKX_t$");
plt.title(r"Estimation of stiff $\mathbf{E}[Y_t^TKX_t|V_t=v]$");
plt.tight_layout()
# save figure
plt.savefig("./fig/cond_exp_examples/oscillator/advection_velocity_estimates.png", dpi=200);

In [ ]:
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
visualize = True
if visualize:
    for i in range(nt):
        plt.figure(1);
        if i % 10 == 0:
            plt.scatter(x[0:idx_mc, i], y[0:idx_mc, i], s=0.5, color="red", label="Data");
            plt.plot(xgrid, y_pred2d[i, :], "--", color="green", lw=2, label=r"NN Predict $\mathbb{E}[X_t^TKY_t|V=v]$");
            plt.xlabel(r"Energy $V$");
            plt.ylabel(r"$X_t^TKY_t$");
            plt.legend();
            plt.ylim([-12000, 12000])
            plt.title(r"$t = {}$".format(i*(t[1]-t[0])));
            display.clear_output(wait=True);
            display.display(pl.gcf());
            plt.clf();
            time.sleep(0.01);

In [ ]:
xgrid

In [ ]:
# save figures
font = {'weight' : 'bold',
        'size'   : 16}
matplotlib.rc('font', **font)
plt.figure(1, figsize=(8, 5));
dt = tgrid[1]-tgrid[0]
plot_t = 0.5
i = int(np.floor(plot_t/dt))
x1 = obs[:, i, 0]
x3 = obs[:, i, 2]
plt.scatter(x1, x3, s=1.5, color="red", label="Data");
plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=2.0, label=r"NN: $\mathbb{E}[X_3(t)|X_1(t)]$");
plt.xlabel(r"$X_1$");
plt.ylabel(r"$X_3$");
plt.title(r"$t = {}$".format(plot_t));
plt.legend(bbox_to_anchor=(0.535, 0.775));
plt.savefig("./fig/cond_exp_examples/kraichnan/nn_estimate_t_{}.png".format(plot_t), dpi=100);

## Experiment 2

The above experiment cannot demonstrate visually the ability of NN to capture highly nonlinear functions. In the following example, we attempt to use the Kraichnan system tested in https://arxiv.org/pdf/1804.02480.pdf

In [ ]:
from PINN.PhysicsInformedROPDF import *
from PINN.utils.dnn import *
# Testing
import matplotlib
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

In [ ]:
# load data
v_path = "../data/LinearOscillator/kraichnan_data.mat"
data = scipy.io.loadmat(v_path)

In [ ]:
obs = data["data"][:, :500, :]
tgrid = data["tspan"].flatten()[:500]
nt = len(tgrid)
nx = obs.shape[0]
X_data = []
y_data = []
for i in range(nt):
    t_i = tgrid[i]
    # append time to all points in x
    inputs_t = torch.tensor([t_i]).repeat(nx).reshape(-1, 1)
    inputs_x = torch.tensor(obs[:, i, 0]).flatten().reshape(-1, 1)
    outputs = torch.tensor(obs[:, i, 2]).flatten().reshape(-1, 1)
    X_i = torch.concat([inputs_t, inputs_x], dim=1)
    # append to full data
    X_data.append(X_i)
    y_data.append(outputs)
X_data = torch.concat(X_data)
y_data = torch.concat(y_data)

def train(inputs, outputs, model, optim, scheduler, batch_size, epochs, shuffle=True):
    X, y = inputs, outputs
    nx = X.shape[0]
    num_batches = int(nx/batch_size)
    
    
    all_losses = []
    for i in range(epochs):
        print("============================================================\n")
        print("Epoch = {}\n".format(i+1));
        print("============================================================\n")
        model.train()
        if shuffle:
            tmp = np.random.permutation(nx)
            X, y = X[tmp, :].data.clone(), y[tmp, :].data.clone()
        for idx in range(num_batches):
            if idx % 100 == 0:
                print("| => | Batch {} |\n".format(idx+1))
        # closure definition
            def closure():
                optim.zero_grad()
                start_idx = idx*batch_size
                end_idx = (idx+1)*batch_size
                if idx + 1 == num_batches:
                    # if last batch
                    end_idx = -1
                Xb, yb = X[start_idx:end_idx, :].data.clone(), y[start_idx:end_idx, :].data.clone()

                # require gradients
                Xb.requires_grad = True
                # make a prediction on the batch
                y_pred = model.forward(Xb)
                # compute L^2 loss
                loss = torch.mean((y_pred - yb)**2)
                # backpropagate
                loss.backward()
                if idx % 100 == 0:
                    print("==> Batch {} loss = {}".format(idx, loss.item()))
                all_losses.append(loss.item())
                return loss
            optim.step(closure=closure)
        if scheduler:
            # step scheduler after epoch if there is one
            scheduler.step()
            print("---------- \n")
            print("++ Learning rate reduced, now at = {}".format(scheduler.get_last_lr()[0]))
    return all_losses

In [ ]:
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
visualize = True
if visualize:
    for i in range(len(tgrid)):
        plt.figure(1);
        x = obs[:, i, 0]
        y = np.log(obs[:, i, 2]) ** 3
        plt.scatter(x, y, s=0.5, color="red", label="Data");
        plt.xlabel(r"$X_1(t)$");
        plt.ylabel(r"$X_3(t)$");
        plt.legend();
        plt.title(r"$t = {}$".format(i*(tgrid[1]-tgrid[0])));
        display.clear_output(wait=True);
        display.display(pl.gcf());
        plt.clf();
        time.sleep(0.05);


In [ ]:
# train using scatter data with Fourier net
nn_fourier2d_cartesian = FourierProductEmbeddedDNN2d(
    layers_time=[20, 20, 20, 1], 
    layers_space=[20, 50, 50, 1], 
    activation=torch.nn.Tanh, 
    last_layer_activation=None, 
    mt=10, 
    mx=10, 
    freq_stds_t=[1.], 
    freq_stds_x=[1.]
)
optim = torch.optim.Adam(
    nn_fourier2d_cartesian.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
train(X_data, y_data, nn_fourier2d_cartesian, optim, scheduler, 2**11, 30, shuffle=True)

In [ ]:
# train using vanilla net
nn_vanilla2d = DNN(layers=[2, 200, 200, 1])
optim = torch.optim.Adam(
    nn_vanilla2d.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
loss_vanilla = train(X_data, y_data, nn_vanilla2d, optim, scheduler, 2**11, 30, shuffle=True)

In [ ]:
xgrid = np.linspace(X_data[:, 1].min(), X_data[:, 1].max(), 2*nx)
xgrid = torch.tensor(xgrid)
tgrid = torch.tensor(tgrid)
# cartesian grid
X = cartesian_data(tgrid, xgrid)
# predict
#y_pred = nn_fourier2d_cartesian(X)
y_pred = nn_vanilla2d(X)

In [ ]:
y_pred2d = y_pred.reshape(2*nx, nt).detach().numpy().T

In [ ]:
plt.imshow(y_pred2d)

In [ ]:
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
visualize = True
if visualize:
    for i in range(len(tgrid)):
        plt.figure(1);
        x1 = obs[:, i, 0]
        x3 = obs[:, i, 2]
        plt.scatter(x1, x3, s=0.5, color="red", label="Data");
        plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=1.0);
        plt.xlabel(r"$X_1(t)$");
        plt.ylabel(r"$X_3(t)$");
        plt.legend();
        plt.title(r"$t = {}$".format(i*(tgrid[1]-tgrid[0])));
        display.clear_output(wait=True);
        display.display(pl.gcf());
        plt.clf();
        time.sleep(0.05);


In [ ]:
# save figures
font = {'weight' : 'bold',
        'size'   : 16}
matplotlib.rc('font', **font)
plt.figure(1, figsize=(8, 5));
dt = tgrid[1]-tgrid[0]
plot_t = 0.5
i = int(np.floor(plot_t/dt))
x1 = obs[:, i, 0]
x3 = obs[:, i, 2]
plt.scatter(x1, x3, s=1.5, color="red", label="Data");
plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=2.0, label=r"NN: $\mathbb{E}[X_3(t)|X_1(t)]$");
plt.xlabel(r"$X_1$");
plt.ylabel(r"$X_3$");
plt.title(r"$t = {}$".format(plot_t));
plt.legend(bbox_to_anchor=(0.535, 0.775));
plt.savefig("./fig/cond_exp_examples/kraichnan/nn_estimate_t_{}.png".format(plot_t), dpi=100);

# save figures
font = {'weight' : 'bold',
        'size'   : 16}
matplotlib.rc('font', **font)
plt.figure(2, figsize=(8, 5));
dt = tgrid[1]-tgrid[0]
plot_t = 1.0
i = int(np.floor(plot_t/dt))
x1 = obs[:, i, 0]
x3 = obs[:, i, 2]
plt.scatter(x1, x3, s=1.5, color="red", label="Data");
plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=2.0, label=r"NN: $\mathbb{E}[X_3(t)|X_1(t)]$");
plt.xlabel(r"$X_1$");
plt.ylabel(r"$X_3$");
plt.title(r"$t = {}$".format(plot_t));
plt.legend(bbox_to_anchor=(0.535, 0.775));
plt.savefig("./fig/cond_exp_examples/kraichnan/nn_estimate_t_{}.png".format(plot_t), dpi=100);

# save figures
font = {'weight' : 'bold',
        'size'   : 16}
matplotlib.rc('font', **font)
plt.figure(3, figsize=(8, 5));
dt = tgrid[1]-tgrid[0]
plot_t = 2.0
i = int(np.floor(plot_t/dt))
x1 = obs[:, i, 0]
x3 = obs[:, i, 2]
plt.scatter(x1, x3, s=1.5, color="red", label="Data");
plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=2.0, label=r"NN: $\mathbb{E}[X_3(t)|X_1(t)]$");
plt.xlabel(r"$X_1$");
plt.ylabel(r"$X_3$");
plt.title(r"$t = {}$".format(plot_t));
plt.legend(bbox_to_anchor=(0.535, 0.775));
plt.savefig("./fig/cond_exp_examples/kraichnan/nn_estimate_t_{}.png".format(plot_t), dpi=100);

font = {'weight' : 'bold',
        'size'   : 16}
matplotlib.rc('font', **font)
plt.figure(4, figsize=(8, 5));
dt = tgrid[1]-tgrid[0]
plot_t = 3.0
i = int(np.floor(plot_t/dt))
x1 = obs[:, i, 0]
x3 = obs[:, i, 2]
plt.scatter(x1, x3, s=1.5, color="red", label="Data");
plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=2.0, label=r"NN: $\mathbb{E}[X_3(t)|X_1(t)]$");
plt.xlabel(r"$X_1$");
plt.ylabel(r"$X_3$");
plt.title(r"$t = {}$".format(plot_t));
plt.legend(bbox_to_anchor=(0.535, 0.775));
plt.savefig("./fig/cond_exp_examples/kraichnan/nn_estimate_t_{}.png".format(plot_t), dpi=100);

## Experiment 3

The value of using an NN is not only learning a continuous model (in time) without tuning parameters as in learning in discrete time using a nonparameteric estimator. I also suspect that NN is capable of learning very complicated regression functions.

In this quick experiment, we engineer a complicated quantity of interest that does not make much physical sense. This is meant to be a test of NN's interpolation capabilities.

... still in progress

In [ ]:
from PINN.PhysicsInformedROPDF import *
from PINN.utils.dnn import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);
# load data
# load data
v_path = "../data/LinearOscillator/kraichnan_data.mat"
data = scipy.io.loadmat(v_path)

In [ ]:
obs = data["data"][:, :500, :]
tgrid = data["tspan"].flatten()[:500]
nt = len(tgrid)
nx = obs.shape[0]
X_data = []
y_data = []
for i in range(nt):
    t_i = tgrid[i]
    # append time to all points in x
    inputs_t = torch.tensor([t_i]).repeat(nx).reshape(-1, 1)
    inputs_x = torch.tensor(obs[:, i, 0]).flatten().reshape(-1, 1)
    
    # create complex quantity of interest to regress
    qoi = ( obs[:, i, 1] ** 3 )
    
    outputs = torch.tensor(qoi).flatten().reshape(-1, 1)
    X_i = torch.concat([inputs_t, inputs_x], dim=1)
    # append to full data
    X_data.append(X_i)
    y_data.append(outputs)
X_data = torch.concat(X_data)
y_data = torch.concat(y_data)

def train(inputs, outputs, model, optim, scheduler, batch_size, epochs, shuffle=True):
    X, y = inputs, outputs
    nx = X.shape[0]
    num_batches = int(nx/batch_size)
    
    
    all_losses = []
    for i in range(epochs):
        print("============================================================\n")
        print("Epoch = {}\n".format(i+1));
        print("============================================================\n")
        model.train()
        if shuffle:
            tmp = np.random.permutation(nx)
            X, y = X[tmp, :].data.clone(), y[tmp, :].data.clone()
        for idx in range(num_batches):
            if idx % 100 == 0:
                print("| => | Batch {} |\n".format(idx+1))
        # closure definition
            def closure():
                optim.zero_grad()
                start_idx = idx*batch_size
                end_idx = (idx+1)*batch_size
                if idx + 1 == num_batches:
                    # if last batch
                    end_idx = -1
                Xb, yb = X[start_idx:end_idx, :].data.clone(), y[start_idx:end_idx, :].data.clone()

                # require gradients
                Xb.requires_grad = True
                # make a prediction on the batch
                y_pred = model.forward(Xb)
                # compute L^2 loss
                loss = torch.mean((y_pred - yb)**2)
                # backpropagate
                loss.backward()
                if idx % 100 == 0:
                    print("==> Batch {} loss = {}".format(idx, loss.item()))
                all_losses.append(loss.item())
                return loss
            optim.step(closure=closure)
        if scheduler:
            # step scheduler after epoch if there is one
            scheduler.step()
            print("---------- \n")
            print("++ Learning rate reduced, now at = {}".format(scheduler.get_last_lr()[0]))
    return all_losses

In [ ]:
# train using vanilla net
nn_vanilla2d = DNN(layers=[2, 200, 200, 1])
optim = torch.optim.Adam(
    nn_vanilla2d.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
loss_vanilla = train(X_data, y_data, nn_vanilla2d, optim, scheduler, 2**11, 30, shuffle=True)

In [ ]:
xgrid = np.linspace(X_data[:, 1].min(), X_data[:, 1].max(), 2*nx)
xgrid = torch.tensor(xgrid)
tgrid = torch.tensor(tgrid)
# cartesian grid
X = cartesian_data(tgrid, xgrid)
# predict
#y_pred = nn_fourier2d_cartesian(X)
y_pred = nn_vanilla2d(X)

In [ ]:
y_pred2d = y_pred.reshape(2*nx, nt).detach().numpy().T
plt.pcolormesh(tgrid, xgrid, y_pred2d.T);

In [ ]:
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
visualize = True
if visualize:
    for i in range(len(tgrid)):
        plt.figure(1);
        x1 = obs[:, i, 0]
        x3 = ( obs[:, i, 1] ** 3 )
        plt.scatter(x1, x3, s=0.5, color="red", label="Data");
        plt.plot(xgrid, y_pred2d[i, :], "--", color="black", lw=1.0);
        plt.xlabel(r"$X_1(t)$");
        plt.ylabel(r"$X_3(t)$");
        plt.legend();
        plt.title(r"$t = {}$".format(i*(tgrid[1]-tgrid[0])));
        display.clear_output(wait=True);
        display.display(pl.gcf());
        plt.clf();
        time.sleep(0.05);
